In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [2]:
import datetime

import torch

import torch.nn as nn

from src.consts import *
from src.main import main, setup_torch, get_corpus
from src.model import RNNModel
from src.training import train, evaluate
from src.split_cross_entropy_loss import SplitCrossEntropyLoss
from src.utils import summary, check_cuda_mem

In [3]:
use_data_paralellization = False

In [4]:
setup_torch()
device = torch.device("cuda" if USE_CUDA else "cpu")
corpus = get_corpus()teste

# TODO remove these two lines
# assert len(corpus.dictionary) == 602755
# assert corpus.valid.size()[0] == 11606861
assert corpus.train.max() < len(corpus.dictionary)
assert corpus.valid.max() < len(corpus.dictionary)
assert corpus.test.max() < len(corpus.dictionary)

ntokens = len(corpus.dictionary)
model = RNNModel(MODEL_TYPE, ntokens, EMBEDDINGS_SIZE, HIDDEN_UNIT_COUNT, LAYER_COUNT, DROPOUT_PROB,
                 TIED)
if use_data_paralellization or USE_DATA_PARALLELIZATION:
    model = CustomDataParallel(model)
else:
    model.to(device)
criterion = nn.CrossEntropyLoss()

summary(model, criterion)

RNNModel(
  (drop): Dropout(p=0.2)
  (encoder): Embedding(16127, 200)
  (rnn): LSTM(200, 200, num_layers=2, dropout=0.2)
  (decoder): Linear(in_features=200, out_features=16127, bias=True)
)

encoder.weight torch.Size([16127, 200])
rnn.weight_ih_l0 torch.Size([800, 200])
rnn.weight_hh_l0 torch.Size([800, 200])
rnn.bias_ih_l0 torch.Size([800])
rnn.bias_hh_l0 torch.Size([800])
rnn.weight_ih_l1 torch.Size([800, 200])
rnn.weight_hh_l1 torch.Size([800, 200])
rnn.bias_ih_l1 torch.Size([800])
rnn.bias_hh_l1 torch.Size([800])
decoder.weight torch.Size([16127, 200])
decoder.bias torch.Size([16127])

Total Parameters: 3,884,727


In [7]:
train(model, corpus, criterion, None, device, False)

INFO 2019-06-14 21:48:34,558: | epoch   1 |   200/ 2965 batches | lr 20.00 | ms/batch  7.47 | loss  4.83 | ppl   125.31
INFO 2019-06-14 21:48:36,006: | epoch   1 |   400/ 2965 batches | lr 20.00 | ms/batch  7.24 | loss  4.86 | ppl   128.84
INFO 2019-06-14 21:48:37,454: | epoch   1 |   600/ 2965 batches | lr 20.00 | ms/batch  7.24 | loss  4.71 | ppl   111.14
INFO 2019-06-14 21:48:38,902: | epoch   1 |   800/ 2965 batches | lr 20.00 | ms/batch  7.24 | loss  4.75 | ppl   115.19
INFO 2019-06-14 21:48:40,351: | epoch   1 |  1000/ 2965 batches | lr 20.00 | ms/batch  7.24 | loss  4.78 | ppl   119.56
INFO 2019-06-14 21:48:41,804: | epoch   1 |  1200/ 2965 batches | lr 20.00 | ms/batch  7.26 | loss  4.77 | ppl   117.75
INFO 2019-06-14 21:48:43,257: | epoch   1 |  1400/ 2965 batches | lr 20.00 | ms/batch  7.26 | loss  4.76 | ppl   116.78
INFO 2019-06-14 21:48:44,711: | epoch   1 |  1600/ 2965 batches | lr 20.00 | ms/batch  7.27 | loss  4.81 | ppl   122.65
INFO 2019-06-14 21:48:46,164: | epoch   